## Cleansing
1. 상장일 2015-12-31 이전 코드만 대상
2. 분석기간동안 거래량 0있는 종목코드 제외


- 종목코드 크롤링

In [0]:
import pandas as pd 

# KRX에서 상장일 정보 가져와서 기존 KOSPI 종목코드 파일과 merge
krx_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0] 

# 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줌 

krx_df.종목코드 = krx_df.종목코드.map('{:06d}'.format) 

# 우리가 필요한 것은 회사명과 종목코드이기 때문에 필요없는 column들은 제외해준다. 
krx_df = krx_df[[ '종목코드', '상장일']] 

# 한글로된 컬럼명을 영어로 바꿔준다. 
# code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code', '상장일':'open_date'}) 


# KOSPI 종목코드 리스트 가져오기
kospi_list = pd.read_excel('kospi_code.xls', sheet_name = 'Sheet1', converters={'종목코드':str})
temp_df = pd.merge(kospi_list, krx_df, on='종목코드', how ='left')

- 상장일이 2015-12-31 이후인 코드는 분석 대상 제외

In [0]:
idx = [1,2,11,3,4]
code_df = temp_df.iloc[:, idx]
stock_df = code_df[code_df['상장일'] < '2015-12-31']
stock_df = stock_df.reset_index(drop = True)

stock_df = stock_df.rename(columns={'종목코드': 'code', '기업명': 'NAME', '상장일':'OPEN_DATE', '업종코드':'BIZ_CD','업종':'BIZ_NM'}) 


stock_df.to_csv("stock_df.csv")

In [0]:
len(stock_df)

726

In [0]:
stock_df.tail(4)

code  NAME   OPEN_DATE  BIZ_CD       BIZ_NM
722  005010   휴스틸  1973-06-29   32401    1차 철강 제조업
723  069260   휴켐스  2002-10-07   32004  기타 화학제품 제조업
724  000540  흥국화재  1974-12-05  116501          보험업
725  003280  흥아해운  1976-06-29   85001       해상 운송업

- Daily 시세정보 가져옴 from 'price_collecting.ipynb'
- 분석기간동안 거래량 0이 있는 종목코드 추출

In [0]:
from pandas import DataFrame

price_df = pd.read_csv("price_df.csv", converters={'code' : str})
out_df = pd.DataFrame()
out_df['code'] = ""
out_df['date'] = ""


for i in range(0,len(price_df)):

    if price_df.iloc[i, 3] == 0:
#             print("거래량 0 발견!",price_df.iloc[i, 4], price_df.iloc[i, 1])
            out_df.loc[len(out_df)] = [price_df.iloc[i, 4], price_df.iloc[i, 1]]
#             out_df.append(pd.DataFrame (data=[[price_df.iloc[i, 4], price_df.iloc[i, 1]] , columns=['code','date']


# 거래량 0인 코드에대한 DF 일단 CSV로 저장!
df2 = out_df.drop_duplicates(['code'])
df2 = df2.reset_index(drop = True)
df2.to_csv("tr_zero_code.csv")

In [0]:
len(df2)

26

In [0]:
tr_zero_code_list = df2['code'].tolist()

drop_list = []
for i in range(0,len(stock_df)):
    
     if stock_df.loc[i, 'code'] in tr_zero_code_list:
            
            drop_list.append(i)
            
stock_df_compl = stock_df.drop(drop_list)
stock_df_compl = stock_df_compl.reset_index(drop = True)

In [0]:
len(stock_df_compl)

700

In [0]:
stock_df_compl.to_csv("stock_df_compl.csv")